In [2]:
import asyncio
import threading
import ccxt
import configparser
import pandas as pd
import time
import warnings

from indicators import supertrend
from utils import check_buy_sell_signals, create_synthetic_pair
from position_manager import Position

pd.set_option('display.max_rows', None)
warnings.filterwarnings('ignore')

config = configparser.ConfigParser()
config.read("config.ini")


exchange = ccxt.kucoin({
  "apiKey": config['KuCoin']['apiKey'],
  "secret": config['KuCoin']['secret'],
  "password": config['KuCoin']['password']
})

In [11]:
# had to make this non-async to work in jupyter
# in main.py
# This is instead of
# async def main()
def main(base_pair=config['Global Settings']['base_pair_default'],
         quote_pair=config['Global Settings']['quote_pair_default']):

    running = True

    while running:
        # This code is for one timeframe. Later, do the same iterated for each timeframe.
        base_bars = exchange.fetch_ohlcv(base_pair, timeframe="1m", limit=50)
        quote_bars = exchange.fetch_ohlcv(quote_pair, timeframe="1m", limit=50)

        pair = create_synthetic_pair(base_bars, quote_bars)

        supertrend_data = supertrend(pair)

        signal = check_buy_sell_signals(supertrend_data)
        print(f"Signal: {signal}")

        if signal.higher == "LONG":
            live_position = Position().open()
            pass
        elif signal.higher == "SHORT":
            live_position = Position().open()
            pass

        # await asyncio.sleep(60)
        time.sleep(60)

# This is how the main function is called in `main.py`:
`asyncio.run(main())`
Maybe this will change later, but it's fine for now.

In [19]:
    ex = ccxt.kucoin
    available_margin = exchange.fetch_borrow_rate('USDT')
    print(available_margin)

NotSupported: kucoinfetchBorrowRate() is not supported yet

When I try to call `exchange.fetch_borrow_rate()` for KuCoin I get the following error:

**NotSupported**: kucoinfetchBorrowRate() is not supported yet

***Traceback***:
```
File ~/Developer/freelance/@N2_Capital/n2_ccxt_supertrend/venv/lib/python3.10/site-packages/ccxt/base/exchange.py:2757, in Exchange.fetch_borrow_rate(self, code, params)
   2755 self.load_markets()
   2756 if not self.has['fetchBorrowRates']:
-> 2757     raise NotSupported(self.id + 'fetchBorrowRate() is not supported yet')
   2758 borrow_rates = self.fetch_borrow_rates(params)
   2759 rate = self.safe_value(borrow_rates, code)
```

So, I created a new file in venv/lib/python3.10/ccxt/mods/base/exchange/fetch_borrow_rate_mod
where to handle this part myself directly with the KuCoin API and return the borrow rate.

***Modified to***:
```
File ~/Developer/freelance/@N2_Capital/n2_ccxt_supertrend/venv/lib/python3.10/site-packages/ccxt/base/exchange.py:2757, in Exchange.fetch_borrow_rate(self, code, params)
    self.load_markets()

    if not self.has['fetchBorrowRates']:
+       from ccxt.mods.base.exchange.fetch_borrow_rate_mod import fetch_borrow_rate_mod
+       borrow_rates = fetch_borrow_rate_mod(params)
+       # I have commented out the NotSupported Error from the standard ccxt library
+       # raise NotSupported(self.id + 'fetchBorrowRate() is not supported yet')
    elif self.has['fetchBorrowRates']:
        borrow_rates = self.fetch_borrow_rates(params)

    rate = self.safe_value(borrow_rates, code)
```

In [19]:
from ccxt.base.exchange import fetch_borrow_rate


In [ ]:
def open_long(self, base_pair, quote_pair, borrow_coin, prompt_borrow: bool, borrow_qty=None):
        # ----------------------------------------
        # Step 1: Query exchange to fetch max
        # [TODO]  borrow quantity of borrow_coin
        #         borrow_coin will be base coin
        #         eg. BTC
        #         of the quote_pair
        #         eg. BTCUSDT


        # available_margin = self.exchange


        # ----------------------------------------
        # Step 2: If prompt_borrow is true, print
        # [TODO]  the max borrow amount retrieved
        #         in step 1, then prompt the user
        #         to either accept the max amount
        #         or instead enter an amount.
        # ----------------------------------------
        # Step 3: Borrow from the exchange in the
        # [TODO]  desired quantity
        # ----------------------------------------
        # Step 4: Sell quote pair using the coins
        # [TODO]  borrowed in step 3.
        #         eg. Sell BTC for USDT
        # ----------------------------------------
        # Step 5: Buy the base coin of the base
        # [TODO]  pair using the quote coin of the
        #         base pair.
        #         eg. Buy ETH with USDT
        # ----------------------------------------
        # Step 6: Generate & Send Email with all
        # [TODO]  of the details about the open
        #         position included.
        pass